<h1>Prophet</h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from numba import jit
import math

In [2]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [3]:
train_1_reduce = train_1[0:100]
train_1_reduce = train_1_reduce.fillna(0)
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
train_1_reduce.head()

Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,EXID_zh.wikipedia.org_all-access_spider,EXO_zh.wikipedia.org_all-access_spider,Energy_zh.wikipedia.org_all-access_spider,Eric_Nam_zh.wikipedia.org_all-access_spider,FAIRY_TAIL_zh.wikipedia.org_all-access_spider,FIESTAR_zh.wikipedia.org_all-access_spider,FIRST_CLASS_zh.wikipedia.org_all-access_spider,Facebook_zh.wikipedia.org_all-access_spider,Fantastic_Duo_zh.wikipedia.org_all-access_spider,Fate/Grand_Order_zh.wikipedia.org_all-access_spider
Dates,,,,,,,,,,,,,,,,,,,,,
2015-07-01,18.0,11.0,1.0,35.0,0.0,12.0,0.0,118.0,5.0,6.0,...,40.0,35.0,16.0,7.0,8.0,9.0,20.0,65.0,0.0,4.0
2015-07-02,11.0,14.0,0.0,13.0,0.0,7.0,0.0,26.0,23.0,3.0,...,9.0,6.0,7.0,5.0,6.0,4.0,9.0,42.0,0.0,9.0
2015-07-03,5.0,15.0,1.0,10.0,0.0,4.0,0.0,30.0,14.0,5.0,...,10.0,41.0,3.0,11.0,10.0,3.0,5.0,30.0,0.0,2.0
2015-07-04,13.0,18.0,1.0,94.0,0.0,5.0,0.0,24.0,12.0,12.0,...,6.0,35.0,3.0,2.0,22.0,7.0,10.0,36.0,0.0,3.0
2015-07-05,14.0,11.0,0.0,4.0,0.0,20.0,0.0,29.0,9.0,6.0,...,28.0,31.0,3.0,7.0,11.0,4.0,7.0,41.0,0.0,2.0


In [4]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [5]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE'])

for column in train_1_reduce.columns:
    print(column)
    df = pd.DataFrame(train_1_reduce[column])
    df = df.reset_index()
    df.columns = ['ds', 'y']
    dftrain = df[0:500]
    dftest = df[500:550]
    m = Prophet()
    m.fit(dftrain)
    future = m.make_future_dataframe(periods=50)
    future.tail()
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    predictions = forecast['yhat'][500:550]
    rmse = np.sqrt(mean_squared_error(dftest['y'], predictions))
    smape = smape_fast(dftest['y'].values, predictions.values)
    print("Test RMSE: %.3f" % rmse)
    print("Test SMAPE: %.3f" % smape)
    metrics_df.loc[column] = [rmse,smape]

2NE1_zh.wikipedia.org_all-access_spider


INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 29.651
Test SMAPE: 44.070
2PM_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 85.190
Test SMAPE: 65.973
3C_zh.wikipedia.org_all-access_spider


...

Test RMSE: 4.445
Test SMAPE: 44.141
Facebook_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 21.749
Test SMAPE: 44.008
Fantastic_Duo_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 12.510
Test SMAPE: 76.820
Fate/Grand_Order_zh.wikipedia.org_all-access_spider
Test RMSE: 38.293
Test SMAPE: 51.180


In [6]:
metrics_df

,RMSE,SMAPE
2NE1_zh.wikipedia.org_all-access_spider,29.650920,44.069894
2PM_zh.wikipedia.org_all-access_spider,85.189853,65.973280
3C_zh.wikipedia.org_all-access_spider,29.410125,55.640537
4minute_zh.wikipedia.org_all-access_spider,8.782234,42.136536
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,26.066972,46.243170
5566_zh.wikipedia.org_all-access_spider,22.874516,51.907413
91Days_zh.wikipedia.org_all-access_spider,24.412211,117.391997
A'N'D_zh.wikipedia.org_all-access_spider,51.867851,74.066307
AKB48_zh.wikipedia.org_all-access_spider,30.012446,37.020266
ASCII_zh.wikipedia.org_all-access_spider,9.353296,24.207763


In [7]:
moyenne_RMSE = metrics_df['RMSE'].mean()
somme_RMSE = metrics_df['RMSE'].sum()
moyenne_SMAPE = metrics_df['SMAPE'].mean()

In [8]:
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)

Test moyenne RMSE: 23.746
Test somme RMSE: 1187.311
Test moyenne SMAPE: 8.768
